<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch13.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 13. Interfaces, Protocols, and ABCs

Program to an interface, not an implementation

Since python 3.8, we have four ways to define an interfaces:
 - Duck typing: Python's default approach
 - Goose typing: supported by abstract base classes (ABCs) which rely on runtime checks of objects against ABCs.
 - Static typing: traditional approach of statically-typed langauges like C and Java
 - Static duck typing: Go langauge approach; supported by subclasses of `typing.Protocol`

## Two Kinds of Protocols

An object protocol specifies methods which an object must provide to fulfill a role.

In [ ]:
class Vowels:
  def __getitem__(self, i):
    return 'AEIOU'[i]

In [ ]:
v = Vowels()
v[0]

'A'

In [ ]:
v[-1]

'U'

In [ ]:
for c in v:
  print(c)

A
E
I
O
U


In [ ]:
'E' in v

True

In [ ]:
'Z' in v

False

Implementing `__getitem__` is enough to allow retrieving items by index, and also to support iteration and the `in` operator.

## Programming Ducks

Sequence and iterable protocols

In summary, given the importance of sequence-like data structures, Python manages to make iteration and the `in` operator work by invoking `__getitem__` when `__iter__` and `__contains` are unavailable

In [ ]:
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

The original `FrenchDeck` does not subclass `abc.Sequence` either, but it does implement both methods of the sequence protocol: `__getitem__` and `__len__`

### Why static type checkers have no chance of dealing with them?

## Monkey Patching: Implementing a Protocol at Runtime

In [ ]:
from random import shuffle

l = list(range(10))
shuffle(l)

In [ ]:
l

[7, 5, 9, 0, 1, 4, 2, 8, 3, 6]

In [ ]:
deck = FrenchDeck()

In [ ]:
shuffle(deck)

In [ ]:
def set_card(deck, position, card):
  deck._cards[position] = card

In [ ]:
FrenchDeck.__setitem__ = set_card

In [ ]:
shuffle(deck)

In [ ]:
deck[:5]

[Card(rank='2', suit='hearts'),
 Card(rank='6', suit='hearts'),
 Card(rank='Q', suit='diamonds'),
 Card(rank='J', suit='clubs'),
 Card(rank='A', suit='hearts')]

## Defensive Programming and "Fail Fast"

In [ ]:
def __init__(self, iterable):
  # don't type check iterable is iterable
  # just use list to see whether iterable is iterable
  self._balls = list(iterable)


## Goose Typing

`isinstance(obj, cls)` is now just fine as long as `cls` is an abstract base class--in other words, `cls`'s metaclass is `abc.ABCMeta`

In [ ]:
class Struggle:
  def __len__(self): return 23

In [ ]:
from collections import abc
isinstance(Struggle(), abc.Sized)

True

In [5]:
from collections import namedtuple, abc

Card = namedtuple('Card', ['rank', 'suit'])

# not check for the implementation of the abstract methods at import time
# but only at runtime when we actually try to instantiate FrenchDeck2
class FrenchDeck2(abc.MutableSequence):
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]

  def __setitem__(self, position, value): # all we need for enabling shuffling
    self._cards[position] = value

  def __delitem__(self, position): # but subclassing MutableSequence forces us to implment __delitem__
    del self._cards[position]

  def insert(self, position, value):
    self._cards.insert(position, value)

#

The use of `isinstance` and `issubclass` becomes more acceptable if you are checking agains ABCs instead of concrete classes. If used with concrete classes, type checks limit polymorphism.

In [2]:
from collections.abc import Sequence
issubclass(FrenchDeck2, Sequence)

True

In [7]:
deck = FrenchDeck2()

In [10]:
for i in reversed(deck):
  print(i)

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')
Card(rank='9', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='7', suit='hearts')
Card(rank='6', suit='hearts')
Card(rank='5', suit='hearts')
Card(rank='4', suit='hearts')
Card(rank='3', suit='hearts')
Card(rank='2', suit='hearts')
Card(rank='A', suit='clubs')
Card(rank='K', suit='clubs')
Card(rank='Q', suit='clubs')
Card(rank='J', suit='clubs')
Card(rank='10', suit='clubs')
Card(rank='9', suit='clubs')
Card(rank='8', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='2', suit='clubs')
Card(rank='A', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(r

To use ABCs well, you need to know what's available. We'll review the `collections` ABCs next.

## ABCs in the Standard Library

The only reliable way to determine whether an object is iterable is to call `iter(obj)`. Same as `hash(obj)`.

Defining and Using an ABC

In [11]:
# tombola.py

import abc

class Tombola(abc.ABC):

  @abc.abstractmethod
  def load(self, iterable):
    """Add items from an iterable"""

  @abc.abstractmethod
  def pick(self):
    """ Remove item at random, returning it

    This method should raise `LookupError` when the instance is empty
    """

  def loaded(self):
    return bool(self.inspect())

  def inspect(self):
    items = []
    while True:
      try:
        items.append(self.pick())
      except LookupError:
        break
    self.load(items)
    return tuple(items)



In [12]:
class Fake(Tombola):
  def pick(self):
    return 13

In [13]:
Fake

__main__.Fake

In [14]:
f = Fake()

TypeError: Can't instantiate abstract class Fake with abstract method load

# Subclassing an ABC

In [15]:
import random

class BingoCage(Tombola):
  def __init__(self, items):
    self._randomizer = random.SystemRandom()
    self._items = []
    self.load(items)

  def load(self, items):
    self._items.extend(items)
    self._randomizer.shuffle(self._items)

  def pick(self):
    try:
      return self._items.pop()
    except IndexError:
      raise LookupError('pick from empty BingoCage')

  def __call__(self):
    self.pick()

In [ ]:
import random

class LottoBlower(Tombola):
  def __init__(self, iterable):
    self._balls = list(iterable)

  def load(self, iterable):
    self._balls.extend(iterable)

  def pick(self):
    try:
      position = random.randrange(len(self._balls))
    except ValueError:
      raise LookupError('pick from empty LottoBlower')

    return self._balls.pop(position)

  def loaded(self):
    return bool(self._balls)

  def inspect(self):
    return tuple(self._balls)

We now come to the crucial dynamic feature of goose typing: declaring virtual subclasses with the `register` method.

## A virtual subclass of an ABC

The registered class becomes a virtual subclass of ABC, and will be recognized as such by `issubclass` but it does not inherit any methods or attributes from the ABC.

In [18]:
from random import randrange

@Tombola.register # Tombolist is registered as a virtual subclass of Tombola
class TomboList(list):
  def pick(self):
    if self:
      position = randrange(len(self))
      return self.pop(position)
    else:
      raise LookupError('pop from empty TomboList')

  load = list.extend

  def loaded(self):
    return bool(self)

  def inspect(self):
    return tuple(self)


In [19]:
issubclass(TomboList, Tombola)

True

In [20]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [22]:
TomboList.__mro__

(__main__.TomboList, list, object)